In [95]:
from os import path
from bs4 import BeautifulSoup
import pandas as pd

METADATA_DESTINATION = '../dataset/imdb_crop/'
METADATA_CROPPED_FILE_MATLAB = METADATA_DESTINATION + 'imdb.mat'

In [96]:
names = open('names.html', 'r', errors='ignore')
actors = open("500actors.txt", 'w')

In [97]:
actors_names = []
soup = BeautifulSoup(names, 'html.parser')
for s in soup.findAll('span'):
    if not('<img' in str(s.contents[0])) and not('<span' in str(s.contents[0])):
        if not(s.contents[0].isdigit()) and not('%' in str(s.contents[0])):
            if s.contents[0] != 'Global Sites' and s.contents[0] != 'Company':
                actors_names.append(s.contents[0])

In [99]:
for a in actors_names:
    actors.write(a + ';')

In [ ]:
actors = open("500actors.txt", 'r')
actors = actors.read().split(';')[:-1]
len(actors)

In [ ]:
head, tail = path.split(METADATA_CROPPED_FILE_MATLAB)
path_serialized = path.join(head, 'imdb.pickle')
df = pd.read_pickle(path_serialized)
df.head()

In [ ]:
df_500actors = pd.DataFrame(columns=df.columns)
for actor in actors:
    q = 'name=="' + actor + '"'
    df_500actors = pd.concat([df_500actors, df.query(q)])
df_500actors.reset_index()

In [ ]:
len(df_500actors)

In [ ]:
head, tail = path.split(METADATA_CROPPED_FILE_MATLAB)
path_serialized = path.join(head, 'imdb_500actors.pickle')
df_500actors.to_pickle(path_serialized)

In [ ]:
df_500actors = pd.read_pickle(path_serialized)
df_500actors.head()

In [ ]:
df_500actors.describe()